In [1]:
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

In [2]:
relevant_columns = [
    "main.disorder",  # Target variable

    # Schizophrenia (alpha PSD)
    "AB.C.alpha.a.FP1", "AB.C.alpha.b.FP2", "AB.C.alpha.c.F7", "AB.C.alpha.d.F3",
    "AB.C.alpha.e.Fz", "AB.C.alpha.f.F4", "AB.C.alpha.g.F8", "AB.C.alpha.h.T3",
    "AB.C.alpha.i.C3", "AB.C.alpha.j.Cz", "AB.C.alpha.k.C4", "AB.C.alpha.l.T4",
    "AB.C.alpha.m.T5", "AB.C.alpha.n.P3", "AB.C.alpha.o.Pz", "AB.C.alpha.p.P4",
    "AB.C.alpha.q.T6", "AB.C.alpha.r.O1", "AB.C.alpha.s.O2",

    # Trauma and stress-related disorders (beta FC)
    "COH.D.beta.a.FP1.b.FP2", "COH.D.beta.c.F7.d.F3", "COH.D.beta.e.Fz.f.F4",
    "COH.D.beta.g.F8.h.T3", "COH.D.beta.i.C3.j.Cz", "COH.D.beta.k.C4.l.T4",
    "COH.D.beta.m.T5.n.P3", "COH.D.beta.o.Pz.p.P4", "COH.D.beta.q.T6.r.O1",

    # Anxiety disorders (whole band PSD: all frequency bands)
    "AB.A.delta.a.FP1", "AB.A.delta.b.FP2", "AB.B.theta.a.FP1", "AB.B.theta.b.FP2",
    "AB.C.alpha.a.FP1", "AB.C.alpha.b.FP2", "AB.D.beta.a.FP1", "AB.D.beta.b.FP2",
    "AB.E.highbeta.a.FP1", "AB.E.highbeta.b.FP2", "AB.F.gamma.a.FP1", "AB.F.gamma.b.FP2",

    # Mood disorders (theta FC)
    "COH.B.theta.a.FP1.b.FP2", "COH.B.theta.c.F7.d.F3", "COH.B.theta.e.Fz.f.F4",
    "COH.B.theta.g.F8.h.T3", "COH.B.theta.i.C3.j.Cz", "COH.B.theta.k.C4.l.T4",
    "COH.B.theta.m.T5.n.P3", "COH.B.theta.o.Pz.p.P4", "COH.B.theta.q.T6.r.O1",

    # Addictive disorders (theta PSD)
    "AB.B.theta.a.FP1", "AB.B.theta.b.FP2", "AB.B.theta.c.F7", "AB.B.theta.d.F3",
    "AB.B.theta.e.Fz", "AB.B.theta.f.F4", "AB.B.theta.g.F8", "AB.B.theta.h.T3",
    "AB.B.theta.i.C3", "AB.B.theta.j.Cz", "AB.B.theta.k.C4", "AB.B.theta.l.T4",
    "AB.B.theta.m.T5", "AB.B.theta.n.P3", "AB.B.theta.o.Pz", "AB.B.theta.p.P4",
    "AB.B.theta.q.T6", "AB.B.theta.r.O1", "AB.B.theta.s.O2",

    # Obsessive-compulsive disorder (gamma FC)
    "COH.F.gamma.a.FP1.b.FP2", "COH.F.gamma.c.F7.d.F3", "COH.F.gamma.e.Fz.f.F4",
    "COH.F.gamma.g.F8.h.T3", "COH.F.gamma.i.C3.j.Cz", "COH.F.gamma.k.C4.l.T4",
    "COH.F.gamma.m.T5.n.P3", "COH.F.gamma.o.Pz.p.P4", "COH.F.gamma.q.T6.r.O1"
]


In [3]:
# Load dataset with only relevant columns
data = pd.read_csv("data/train.csv", usecols=relevant_columns)

# Filter only Mood Disorder and Healthy Control
data = data[data["main.disorder"].isin(["Healthy control", "Addictive disorder"])]

# Encode labels manually: Healthy Control -> 0, Mood Disorder -> 1
data["main.disorder"] = data["main.disorder"].map({
    "Healthy control": 0,
    "Addictive disorder": 1
})

# Features and labels
X = data.drop(columns=["main.disorder"]).values
y = data["main.disorder"].values.astype(int)  # Ensure integer labels

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save the scaler
joblib.dump(scaler, "svm_addict_scaler.pkl")

# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [4]:
# ---------------- Hyperparameter Tuning (Grid Search) ----------------
param_grid = {
    "C": [0.1, 0.5, 1, 5, 10],  # Regularization parameter
    "kernel": ["linear", "rbf"],  # Linear and RBF kernel
}

svm = SVC(probability=True, class_weight="balanced", random_state=42)

grid_search = GridSearchCV(svm, param_grid, cv=5, scoring="roc_auc", n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Get the best model
best_svm = grid_search.best_estimator_

# Save the trained SVM model
joblib.dump(best_svm, "svm_addict_tuned.pkl")

Fitting 5 folds for each of 10 candidates, totalling 50 fits


['svm_addict_tuned.pkl']

In [9]:
# ---------------- Evaluate Best Model ----------------
y_pred = best_svm.predict(X_test)
y_pred_prob = best_svm.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_prob)

print(f"\n Tuned SVM Model for Trauma Disorder vs. Healthy Control")
print(f" Best Parameters: {grid_search.best_params_}")
print(f" Accuracy: {accuracy:.4f}")
print(f" AUC Score: {auc:.4f}")


 Tuned SVM Model for Trauma Disorder vs. Healthy Control
 Best Parameters: {'C': 10, 'kernel': 'rbf'}
 Accuracy: 0.5000
 AUC Score: 0.4906


In [13]:
def predict_mood_disorder(new_data_path):
    """
    Loads new EEG data, scales it, and predicts Mood Disorder probability.
    """
    # Load new data
    new_data = pd.read_csv(new_data_path, usecols=relevant_columns[1:])  # Exclude "main.disorder"
    
    # Load saved scaler and model
    scaler = joblib.load("svm_addict_scaler.pkl")
    svm = joblib.load("svm_addict_tuned.pkl")

    # Load new data
    new_data = pd.read_csv(new_data_path, usecols=relevant_columns[1:])  # Exclude "main.disorder"

# Convert DataFrame to NumPy before applying StandardScaler
    new_data_scaled = scaler.transform(new_data.values)  # <-- FIXED


    # Predict probabilities
    predictions = svm.predict_proba(new_data_scaled)[:, 1]  # Probability of Mood Disorder

    return predictions

In [14]:
preds = predict_mood_disorder("data.csv")


In [15]:
from sklearn.metrics import roc_curve

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Find the optimal threshold (Youden’s J statistic: max(TPR - FPR))
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

print(f"🔹 Optimal Decision Threshold: {optimal_threshold:.4f}")

# Convert probabilities to labels using the optimal threshold
pred_labels = (y_pred_prob >= optimal_threshold).astype(int)


🔹 Optimal Decision Threshold: 0.7445


In [16]:
print(preds)

[0.72845777 0.7087211  0.69321872 0.69350327 0.64210602 0.44354004
 0.7949923  0.61991721 0.68042164 0.4607295  0.76408364 0.76119109
 0.67464796 0.71140553 0.58107953 0.82639099 0.74220143 0.67434394
 0.82545698 0.78836415 0.5        0.64059332 0.63844083 0.77131275
 0.82226958 0.68030624 0.64737466 0.76208576 0.71606733 0.6938843
 0.58917904 0.79966792 0.58126675 0.69302269 0.54252774 0.70222418
 0.73324888 0.65073696 0.71289627 0.5684412  0.82921376 0.75656954
 0.68097579 0.72543195 0.78520097 0.67878564 0.78582144 0.87188607
 0.41002708 0.6560393  0.64796182 0.70366987 0.70310913 0.74685128
 0.6369752  0.60951981 0.7408852  0.73302703 0.5        0.74038924
 0.71168765 0.71450872 0.47224907 0.70922858 0.69001527 0.7109336
 0.5        0.77361795 0.75267754 0.60295149 0.70553003 0.63745699
 0.64727806 0.76499913 0.67273533 0.79215979 0.57428631 0.76431655
 0.68758271 0.7701906  0.48349877 0.75282904 0.80225682 0.67260821
 0.55968444 0.7849671  0.61641706 0.76201505 0.68543128 0.657540

In [17]:
def decode_predictions(preds):
    """
    Convert predictions (0,1) back to disorder labels.
    """
    label_map = {0: "Healthy control", 1: "Addictive disorder"}
    return [label_map[pred] for pred in preds]

# Example usage
pred_labels = (preds >= 0.7445).astype(int)  # Convert probabilities to binary labels
decoded_labels = decode_predictions(pred_labels)

print("\nPredicted Labels:\n", decoded_labels)


Predicted Labels:
 ['Healthy control', 'Healthy control', 'Healthy control', 'Healthy control', 'Healthy control', 'Healthy control', 'Addictive disorder', 'Healthy control', 'Healthy control', 'Healthy control', 'Addictive disorder', 'Addictive disorder', 'Healthy control', 'Healthy control', 'Healthy control', 'Addictive disorder', 'Healthy control', 'Healthy control', 'Addictive disorder', 'Addictive disorder', 'Healthy control', 'Healthy control', 'Healthy control', 'Addictive disorder', 'Addictive disorder', 'Healthy control', 'Healthy control', 'Addictive disorder', 'Healthy control', 'Healthy control', 'Healthy control', 'Addictive disorder', 'Healthy control', 'Healthy control', 'Healthy control', 'Healthy control', 'Healthy control', 'Healthy control', 'Healthy control', 'Healthy control', 'Addictive disorder', 'Addictive disorder', 'Healthy control', 'Healthy control', 'Addictive disorder', 'Healthy control', 'Addictive disorder', 'Addictive disorder', 'Healthy control', 'He

In [18]:
import pandas as pd

# Convert predictions into a DataFrame
output_df = pd.DataFrame({"Predicted Label": decoded_labels})

# Save to CSV
output_df.to_csv("predicted_addict_labels.csv", index=False)

print(" Predictions saved to 'predicted_labels.csv'")


 Predictions saved to 'predicted_labels.csv'
